# Random grammar generation

# Creates a scale-free random grammar with specified parameters

We start by obtaining the parameters of the grammar

In [1]:
# Sliders for grammar parameters
from ipywidgets import interact, interactive, fixed, interact_manual

def GetParams(num_words, num_classes, num_class_connectors, connectors_limit):
    return (num_words, num_classes, num_class_connectors, connectors_limit)
grammar_params = interactive(GetParams, 
                             num_words = (15, 34), # vocabulary size
                             num_classes = (3, 5), # number of grammar classes
                             num_class_connectors = (5, 11), # connectors between grammar classes (directed)
                             connectors_limit = (1, 3)) # max connectors a word can have
display(grammar_params)

interactive(children=(IntSlider(value=24, description='num_words', max=34, min=15), IntSlider(value=4, descrip…

In [42]:
# Set Grammar parameters
num_words = grammar_params.result[0]
num_classes = grammar_params.result[1]
num_class_connectors = grammar_params.result[2]
connectors_limit = grammar_params.result[3]

In [43]:
# Populate grammar classes following a Zipf distribution
from fractions import Fraction
import numpy as np

harmonic_number = sum(Fraction(1, d) for d in range(1, num_classes + 1))

zipf_fracs = [1 / x / harmonic_number for x in range(1, num_classes + 1)]

words_per_class = np.array(np.round(np.array(zipf_fracs) * num_words), dtype = "int")
cumul_words = np.cumsum(words_per_class) # boundaries for class words
print(cumul_words)

[12 18 22 25]


In [44]:
# Plot vocab fraction vs rank for grammar classes
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)         # initiate notebook for offline plot

class_freqs = Scatter(
    x = np.array(range(1, num_classes + 1)),
    y = words_per_class / num_words
)
Zipf_law = Scatter(
    x = np.array(range(1, num_classes + 1)),
    y = zipf_fracs
)
data = [class_freqs, Zipf_law]
iplot(data)

In [45]:
# Create random connectors between grammar classes
import random as rand
connectors = set()
for i in range(0, num_class_connectors - 1):
    randint1 = rand.randint(0, num_classes - 1)
    randint2 = rand.randint(0, num_classes - 1)
    if randint1 != randint2: # avoid classes connecting to themselves; may cause less connections than param
        connectors.add((randint1, randint2))

#connectors = set([(rand.randint(0, num_classes - 1), rand.randint(0, num_classes - 1)) for i in range(0, num_class_connectors)])
print(connectors)

{(1, 2), (3, 1), (2, 1), (2, 0), (2, 3), (1, 0), (0, 3)}


In [46]:
# Assign connectors to classes
# Translate connectors into connector labels
connectors_dict = {k:[] for k in range(num_classes)}
connectors_dict_text = {k:[] for k in range(num_classes)}
for connector in connectors:
    connector_text = "C" + str(connector[0]) + "_" + str(connector[1])
    connectors_dict[connector[0]].append(connector)
    connectors_dict[connector[1]].append(connector)
    connectors_dict_text[connector[0]].append(connector_text + "+")
    connectors_dict_text[connector[1]].append(connector_text + "-")
    
print(connectors_dict_text)
print(connectors_dict)

{0: ['C2_0-', 'C1_0-', 'C0_3+'], 1: ['C1_2+', 'C3_1-', 'C2_1-', 'C1_0+'], 2: ['C1_2-', 'C2_1+', 'C2_0+', 'C2_3+'], 3: ['C3_1+', 'C2_3-', 'C0_3-']}
{0: [(2, 0), (1, 0), (0, 3)], 1: [(1, 2), (3, 1), (2, 1), (1, 0)], 2: [(1, 2), (2, 1), (2, 0), (2, 3)], 3: [(3, 1), (2, 3), (0, 3)]}


In [47]:
# Build the disjuncts randomly, with some directives
dict_disjuncts = {}
for gramm_class, connects in connectors_dict.items():
     # don't conjunct more connectors than available ones, nor than limit
    max_connectors = min(connectors_limit, len(connects))
    disjuncts = []
    
    for connector in connects: # create one conjunct per connector; arbitrary choice
        num_connectors = rand.randint(1, max_connectors) # determine how many connectors for this conjunct
        conjunct = [connector] # current connector always goes in conjunct
        
        diff_connects = connects[:] # make independent copy
        diff_connects.remove(connector) # don't repeat connector in a conjunct
        conjunct.extend(rand.sample(diff_connects, num_connectors - 1)) # add random connectors to conjunct; no repeats
            
        disjuncts.append(conjunct)
        
    dict_disjuncts[gramm_class] = set(tuple(d) for d in disjuncts) # set eliminates duplicate disjuncts

print(dict_disjuncts)

{0: {((2, 0), (1, 0)), ((1, 0),), ((0, 3),)}, 1: {((1, 0), (2, 1)), ((2, 1),), ((1, 2), (1, 0)), ((3, 1), (2, 1))}, 2: {((1, 2), (2, 0)), ((2, 0), (1, 2)), ((2, 1),), ((2, 3),)}, 3: {((3, 1), (2, 3)), ((0, 3), (3, 1)), ((2, 3),)}}


In [48]:
# Translate grammar to dictionary format
grammar_text = f"""
% RANDOM GRAMMAR with parameters:
% num_words = {num_words}
% num_classes = {num_classes}
% num_class_connectors = {num_class_connectors}
% connectors_limit = {connectors_limit}
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

"""

for curr_class, disjunct in dict_disjuncts.items():
    class_entry = f"% Class: {curr_class}\n" # Description of current disjunct
    # Calculate initial word_id for class
    if curr_class == 0:
        lower_id = 0
    else:
        lower_id = cumul_words[curr_class - 1]
        
    # Add word list to class_entry
    class_words = [f'"W{i}_{curr_class}"' for i in range(lower_id, cumul_words[curr_class])]
    class_entry += ", ".join(class_words) + ":\n"
    
    # Add every conjunct to disjunct
    curr_disjunct = []
    for conjunct in disjunct:
        curr_conjunct = []
        for connector in conjunct:
            connector_text = "C" + str(connector[0]) + "_" + str(connector[1])
            sign = "+" if connector[0] == curr_class else "-" # choose connector sign
            curr_conjunct.append(connector_text + sign)
        curr_disjunct.append(" & ".join(curr_conjunct))
        
    class_entry += "(" + ") or (".join(curr_disjunct) + ");\n\n"
    grammar_text += class_entry
    
print(grammar_text)


% RANDOM GRAMMAR with parameters:
% num_words = 24
% num_classes = 4
% num_class_connectors = 10
% connectors_limit = 2
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

% Class: 0
"W0_0", "W1_0", "W2_0", "W3_0", "W4_0", "W5_0", "W6_0", "W7_0", "W8_0", "W9_0", "W10_0", "W11_0":
(C2_0- & C1_0-) or (C1_0-) or (C0_3+);

% Class: 1
"W12_1", "W13_1", "W14_1", "W15_1", "W16_1", "W17_1":
(C1_0+ & C2_1-) or (C2_1-) or (C1_2+ & C1_0+) or (C3_1- & C2_1-);

% Class: 2
"W18_2", "W19_2", "W20_2", "W21_2":
(C1_2- & C2_0+) or (C2_0+ & C1_2-) or (C2_1+) or (C2_3+);

% Class: 3
"W22_3", "W23_3", "W24_3":
(C3_1+ & C2_3-) or (C0_3- & C3_1+) or (C2_3-);




# As grammar is ready, let's generate a corpus from it

In [49]:
import collections
import random

class GrammarSampler(object):
    def __init__(self, grammar, cumul_words):
        self.grammar = grammar
        self.word_dist = cumul_words
        self.word_dist = np.insert(self.word_dist, 0, 0) # help to select words
        self.counter = 0
        self.links = {}
        self.sentence = None
        self.flatTree = None
        self.ullParse = None
        self.ullLinks = []
        
    def ReturnPos(self, word_string):
        """
        Given a word string, find its position in the tree
        """
        split_word = word_string.split("_")
        word_tuple = (int(split_word[2]), int(split_word[1]))
        return self.flatTree.index(word_tuple)

    def GenerateParse(self):
        """
        Generate a lexical tree and return an its corresponding sentence
        """
        # First generate a random tree
        tree_sample = self.GenerateTree()
        print(f"Tree:\n{tree_sample}")
        self.flatTree = list(self.Flatten(tree_sample))
        print(f"Flat tree: {self.flatTree}")
        
        # Obtain links from random tree
        self.ConstructLinks(tree_sample)
        sentence_array = np.full(len(self.flatTree), None) # initialize empty sentence array
        
        # Fill sentence array
        for key, value in self.links.items():
            key_pos = self.ReturnPos(key) # search for word-instance position in the tree
            sentence_array[key_pos] = key 
            for val in value:
                val_pos = self.ReturnPos(val)
                sentence_array[val_pos] = val
                self.ullLinks.append(f"{key_pos} {key} {val_pos} {val}")
                
        # Concatenate parse text output
        self.sentence = " ".join(sentence_array)
        self.ullParse = f"{self.sentence}\n" + "\n".join(self.ullLinks)
        print(f"Sentence: {self.sentence}")
        print(f"ULL parse: \n{self.ullParse}")
        
    def Flatten(self, l): # taken from https://stackoverflow.com/questions/2158395/flatten-an-irregular-list-of-lists
        """
        Given a list of nested lists, returns a flat structure in the same sequence.
        Useful to find the order of words in a sentence
        """
        for el in l:
            if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes, tuple)):
                yield from self.Flatten(el)
            else:
                yield el    
    
    def ChooseConjunct(self, connector, disjunct):
        """
        Chooses a random conjunct from the ones in disjunct that contain connector
        """
        valid_conjs = [conj for conj in disjunct if connector in conj] # filters inappropriate connectors
        return list(rand.choice(valid_conjs))

    def GenerateTree(self, node_class = None, connector = ()):
        """ 
        Generate a random tree of class elements from the
        grammar, starting with the given class.
        """
        if self.counter == 0: # handle initial case
            node_class = rand.randint(0, len(self.grammar) - 1)
            print(f"Seed node_class: {node_class}")
            conjunct = rand.sample(self.grammar[node_class], 1)[0]
        else:
            # select one valid production of this class randomly
            conjunct = self.ChooseConjunct(connector, self.grammar[node_class])
            conjunct.remove(connector) # eliminate connector already used

        tree = [(self.counter, node_class)] # leaf tuple structure: (word_order, class)

        # for non-terminals, recurse
        for conn in conjunct:
            self.counter += 1
            new_node_class = list(conn)
            new_node_class.remove(node_class)
            # determine to insert subtree at beginning or end of current one
            insert_pos = len(tree) if conn.index(node_class) == 0 else 0
            tree.insert(insert_pos, self.GenerateTree(new_node_class[0], conn))
            
        return tree
    
    def SampleWord(self, pos, grammar_class):
        """
        Samples word from given grammar_class, and returns string in format
        "Wa_b_c", where a is the word number, b is the word class, c is word's position
        in the sentence
        """
        chosen_word = rand.randint(self.word_dist[grammar_class], self.word_dist[grammar_class + 1] - 1)
        word_string = "W" + str(chosen_word) + f"_{grammar_class}_" + str(pos)
        return word_string
        
    def ConstructLinks(self, tree):
        """
        Given a tree, store the links it contains in self.links
        """
        curr_word_class = [i for i in tree if isinstance(i, tuple)][0]
        curr_string = self.SampleWord(curr_word_class[0], curr_word_class[1]) # Samples word in string format
        
        # non-terminal case
        if len(tree) > 1:
            self.links[curr_string] = []
            tree.remove(curr_word_class)
            for subtree in tree:
                self.links[curr_string].append(self.ConstructLinks(subtree))
                
        return curr_string

In [56]:
print(dict_disjuncts)
print(grammar_text)
test = GrammarSampler(dict_disjuncts, cumul_words)
test.GenerateParse()

{0: {((2, 0), (1, 0)), ((1, 0),), ((0, 3),)}, 1: {((1, 0), (2, 1)), ((2, 1),), ((1, 2), (1, 0)), ((3, 1), (2, 1))}, 2: {((1, 2), (2, 0)), ((2, 0), (1, 2)), ((2, 1),), ((2, 3),)}, 3: {((3, 1), (2, 3)), ((0, 3), (3, 1)), ((2, 3),)}}

% RANDOM GRAMMAR with parameters:
% num_words = 24
% num_classes = 4
% num_class_connectors = 10
% connectors_limit = 2
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

% Class: 0
"W0_0", "W1_0", "W2_0", "W3_0", "W4_0", "W5_0", "W6_0", "W7_0", "W8_0", "W9_0", "W10_0", "W11_0":
(C2_0- & C1_0-) or (C1_0-) or (C0_3+);

% Class: 1
"W12_1", "W13_1", "W14_1", "W15_1", "W16_1", "W17_1":
(C1_0+ & C2_1-) or (C2_1-) or (C1_2+ & C1_0+) or (C3_1- & C2_1-);

% Class: 2
"W18_2", "W19_2", "W20_2", "W21_2":
(C1_2- & C2_0+) or (C2_0+ & C1_2-) or (C2_1+) or (C2_3+);

% Class: 3
"W22_3", "W23_3", "W24_3":
(C3_1+ & C2_3-) or (C0_3- & C3_1+) or (C2_3-);


Seed node_class: 2
Tree:
[[(1, 1), [(2, 0)]], (0, 2), [[(4, 1), [(5, 2), [[[(8, 2)], (7, 1)], (6, 0)]]], (3, 0)]]
Flat tree